In [3]:
from datasets import Dataset
from transformers import (
    GPT2Tokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments,
    DataCollatorForLanguageModeling
)

# Шаг 1: Загрузка модели и токенизатора
model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Добавляем pad_token (у GPT2 его нет по умолчанию)
tokenizer.pad_token = tokenizer.eos_token

# Шаг 2: ТЕСТ ДО ДООБУЧЕНИЯ
print("=== BEFORE FINE-TUNING ===")
test_prompts = [
    "I need your help with",
    "Can you explain",
    "Tell me about"
]

for prompt in test_prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs, 
        max_length=30, 
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    print(f"Prompt: {prompt}")
    print(f"Response: {tokenizer.decode(outputs[0], skip_special_tokens=True)}\n")

# Шаг 3: Подготовка датасета для уважительных ответов
polite_data = {
    'text': [
        "I need your help with this task. I would be happy to assist you with that task.",
        "Can you explain this concept? Certainly, I'd be delighted to explain this concept to you.",
        "Tell me about artificial intelligence. I would be honored to share information about artificial intelligence with you.",
        "What is machine learning? I'm pleased to explain machine learning to you in detail.",
        "How does this work? I'd be glad to help you understand how this works.",
        "I don't understand. Please allow me to clarify that for you respectfully.",
        "Can you help me? Of course, I would be honored to help you with your request.",
        "Explain the process. I'm happy to walk you through the process step by step.",
        "What should I do? I respectfully suggest considering the following approach.",
        "Is this correct? Allow me to provide you with a thoughtful answer on this matter."
    ]
}

dataset = Dataset.from_dict(polite_data)

# Шаг 4: Токенизация датасета
def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        truncation=True, 
        padding='max_length', 
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# Шаг 5: Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False  # Causal LM, не masked LM
)

# Шаг 6: Training Arguments
training_args = TrainingArguments(
    output_dir="./polite_distilgpt2",
    num_train_epochs=15,
    per_device_train_batch_size=2,
    save_steps=500,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=100
)

# Шаг 7: Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# Шаг 8: Обучение модели
print("\n=== STARTING TRAINING ===")
trainer.train()

# Шаг 9: Сохранение модели
model.save_pretrained("./polite_distilgpt2_final")
tokenizer.save_pretrained("./polite_distilgpt2_final")

# Шаг 10: ТЕСТ ПОСЛЕ ДООБУЧЕНИЯ
print("\n=== AFTER FINE-TUNING ===")
for prompt in test_prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs, 
        max_length=30, 
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    print(f"Prompt: {prompt}")
    print(f"Response: {tokenizer.decode(outputs[0], skip_special_tokens=True)}\n")


=== BEFORE FINE-TUNING ===
Prompt: I need your help with
Response: I need your help with this project.


If you have any questions about this project, please contact me.
If you have anything else

Prompt: Can you explain
Response: Can you explain the topic of that article.


“The question of the “toughness for education“ is very important

Prompt: Tell me about
Response: Tell me about it, and let me know what you think about her.”



Map: 100%|██████████| 10/10 [00:00<00:00, 984.49 examples/s]



=== STARTING TRAINING ===


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss



=== AFTER FINE-TUNING ===
Prompt: I need your help with
Response: I need your help with this request. I would be happy to assist you with that request. I would be happy to assist you with that request.

Prompt: Can you explain
Response: Can you explain this concept? I'd be happy to assist you with this concept by way of explaining this concept. I'd be happy to help you

Prompt: Tell me about
Response: Tell me about artificial intelligence. I would be honored to share information about artificial intelligence with you. I would be honored to share information about artificial intelligence with

